# 0. Setup

In [1]:
import pandas as pd
import seaborn as sns
import re

# 1. Data Extraction

We filter for the county that was assigned for analysis and the required date range.

Name: Salt Lake County

Date range: Feb 1, 2020 to Oct 1, 2021

## 1.1 Load data from CSV sources

In [2]:
covid_cases = pd.read_csv("RAW_us_confirmed_cases.csv")
mask_use = pd.read_csv("mask-use-by-county.csv")
mask_mandate = pd.read_csv("US_State_and_Territorial_Public_Mask_Mandates_From_April_10__2020_through_August_15__2021_by_County_by_Day.csv")

In [3]:
covid_cases.head()

,Province_State,Admin2,UID,iso2,iso3,code3,FIPS,Country_Region,Lat,Long_,...,10/21/22,10/22/22,10/23/22,10/24/22,10/25/22,10/26/22,10/27/22,10/28/22,10/29/22,10/30/22
0,Alabama,Autauga,84001001,US,USA,840,1001.0,US,32.539527,-86.644082,...,18480,18480,18480,18480,18480,18480,18511,18511,18511,18511
1,Alabama,Baldwin,84001003,US,USA,840,1003.0,US,30.727750,-87.722071,...,65895,65895,65895,65895,65895,65895,65973,65973,65973,65973
2,Alabama,Barbour,84001005,US,USA,840,1005.0,US,31.868263,-85.387129,...,6926,6926,6926,6926,6926,6926,6930,6930,6930,6930
3,Alabama,Bibb,84001007,US,USA,840,1007.0,US,32.996421,-87.125115,...,7560,7560,7560,7560,7560,7560,7575,7575,7575,7575
4,Alabama,Blount,84001009,US,USA,840,1009.0,US,33.982109,-86.567906,...,17286,17286,17286,17286,17286,17286,17320,17320,17320,17320


In [4]:
mask_use.head()

,COUNTYFP,NEVER,RARELY,SOMETIMES,FREQUENTLY,ALWAYS
0,1001,0.053,0.074,0.134,0.295,0.444
1,1003,0.083,0.059,0.098,0.323,0.436
2,1005,0.067,0.121,0.120,0.201,0.491
3,1007,0.020,0.034,0.096,0.278,0.572
4,1009,0.053,0.114,0.180,0.194,0.459


In [5]:
mask_mandate.head()

,State_Tribe_Territory,County_Name,FIPS_State,FIPS_County,date,order_code,Face_Masks_Required_in_Public,Source_of_Action,URL,Citation
0,AL,Autauga County,1,1,4/10/2020,2,NaN,NaN,NaN,NaN
1,AL,Autauga County,1,1,4/11/2020,2,NaN,NaN,NaN,NaN
2,AL,Autauga County,1,1,4/12/2020,2,NaN,NaN,NaN,NaN
3,AL,Autauga County,1,1,4/13/2020,2,NaN,NaN,NaN,NaN
4,AL,Autauga County,1,1,4/14/2020,2,NaN,NaN,NaN,NaN


## 1.2 Filter data for Salt Lake County, Utah

In [6]:
# Extracting data for Salt Lake County, Utah

covid_cases = covid_cases.loc[(covid_cases['Province_State'] == 'Utah') & (covid_cases['Admin2'] == 'Salt Lake')]
covid_cases.head()

,Province_State,Admin2,UID,iso2,iso3,code3,FIPS,Country_Region,Lat,Long_,...,10/21/22,10/22/22,10/23/22,10/24/22,10/25/22,10/26/22,10/27/22,10/28/22,10/29/22,10/30/22
2976,Utah,Salt Lake,84049035,US,USA,840,49035.0,US,40.666165,-111.921601,...,392045,392045,392045,392045,392045,392045,392864,392864,392864,392864


We find that the FIPS code for the county is 49035. Only one row of data was returned but each day's case count is a separate column. We will have to clean that before we can proceed.

In [7]:
# Extracting mask use data
mask_use = mask_use.loc[mask_use['COUNTYFP'] == 49035]
mask_use.head()

,COUNTYFP,NEVER,RARELY,SOMETIMES,FREQUENTLY,ALWAYS
2794,49035,0.028,0.032,0.094,0.202,0.644


In [8]:
# Extracting mask mandate data
mask_mandate =  mask_mandate.loc[(mask_mandate["FIPS_State"] == 49) & (mask_mandate["FIPS_County"] == 35)]
mask_mandate.head()

,State_Tribe_Territory,County_Name,FIPS_State,FIPS_County,date,order_code,Face_Masks_Required_in_Public,Source_of_Action,URL,Citation
743140,UT,Salt Lake County,49,35,4/13/2020,2,NaN,NaN,NaN,NaN
743177,UT,Salt Lake County,49,35,4/14/2020,2,NaN,NaN,NaN,NaN
743206,UT,Salt Lake County,49,35,4/10/2020,2,NaN,NaN,NaN,NaN
743207,UT,Salt Lake County,49,35,4/11/2020,2,NaN,NaN,NaN,NaN
743208,UT,Salt Lake County,49,35,4/12/2020,2,NaN,NaN,NaN,NaN


## 1.3 Filter data for Feb 1, 2020 to Oct 1, 2021

In [9]:
# Pivot the data
date_cols = [col for col in covid_cases.columns if re.search(r'(\d{1,2}/)+\d{2}', col)]
covid_cases = covid_cases.melt(id_vars=['Admin2', 'Province_State',], value_vars=date_cols, var_name='Date', value_name='Cases')
covid_cases["Date"] = pd.to_datetime(covid_cases["Date"])
covid_cases = covid_cases[(covid_cases["Date"] >= "2020-02-01") & (covid_cases["Date"] <= "2021-10-01")]
covid_cases.head()

,Admin2,Province_State,Date,Cases
10,Salt Lake,Utah,2020-02-01,0
11,Salt Lake,Utah,2020-02-02,0
12,Salt Lake,Utah,2020-02-03,0
13,Salt Lake,Utah,2020-02-04,0
14,Salt Lake,Utah,2020-02-05,0


In [10]:
covid_cases.describe(include='all', datetime_is_numeric=True)

,Admin2,Province_State,Date,Cases
count,609,609,609,609.000000
unique,1,1,NaN,NaN
top,Salt Lake,Utah,NaN,NaN
freq,609,609,NaN,NaN
mean,NaN,NaN,2020-12-01 00:00:00,81164.729064
min,NaN,NaN,2020-02-01 00:00:00,0.000000
25%,NaN,NaN,2020-07-02 00:00:00,11728.000000
50%,NaN,NaN,2020-12-01 00:00:00,81337.000000
75%,NaN,NaN,2021-05-02 00:00:00,148226.000000
max,NaN,NaN,2021-10-01 00:00:00,183169.000000


In [12]:
mask_mandate["date"] = pd.to_datetime(mask_mandate["date"])
mask_mandate = mask_mandate[(mask_mandate["date"] >= "2020-02-01") & (mask_mandate["date"] <= "2021-10-01")]
mask_mandate.head()

,State_Tribe_Territory,County_Name,FIPS_State,FIPS_County,date,order_code,Face_Masks_Required_in_Public,Source_of_Action,URL,Citation
743140,UT,Salt Lake County,49,35,2020-04-13,2,NaN,NaN,NaN,NaN
743177,UT,Salt Lake County,49,35,2020-04-14,2,NaN,NaN,NaN,NaN
743206,UT,Salt Lake County,49,35,2020-04-10,2,NaN,NaN,NaN,NaN
743207,UT,Salt Lake County,49,35,2020-04-11,2,NaN,NaN,NaN,NaN
743208,UT,Salt Lake County,49,35,2020-04-12,2,NaN,NaN,NaN,NaN
